<a href="https://colab.research.google.com/github/riesacassano/fMRI_piano/blob/master/ryc_SRM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install brainiak

In [ ]:
import numpy as np
from scipy import stats
from scipy.io import loadmat
from brainiak.funcalign.srm import SRM
from brainiak.isc import isc
import matplotlib.pyplot as plt

In [5]:
no_TRs = 148
no_conds = 4
no_runs = 3

Load data for each subject in the selected ROI.

In [6]:
roi = 'PMC' #'A1', 'MotorCortex', 'PMC'
filepath = 'drive/MyDrive/fMRI_music_data/%s/'%roi
sub = ['03','15','20','23']

In [20]:
# load in the training data as a list of 2D arrays
train_data = []
for s in range(len(sub)):
  train_data.append(np.load(filepath+'sub1%s_runs23.npy'%sub[s]))

# load in test data as a list of 2D arrays
test_data = []
for s in range(len(sub)):
  test_data.append(np.load(filepath+'sub1%s_run1.npy'%sub[s]))

In [8]:
print(train_data[0].shape)
print(test_data[0].shape)

(1573, 1184)
(1573, 592)


In [9]:
# normalize the data by subject
for s in range(len(sub)):
  train_data[s] = stats.zscore(train_data[s],axis=1,ddof=1) 
  # not sure why we need the DOF correction here
  train_data[s] = np.nan_to_num(train_data[s]) # or this...
  
  test_data[s] = stats.zscore(test_data[s],axis=1,ddof=1)
  test_data[s] = np.nan_to_num(test_data[s])

Fit SRM


(code borrowed from BrainIAK tutorial 11 - SRM)

In [10]:
features = 30  # How many features will you fit?
n_iter = 20  # How many iterations of fitting will you perform

# Create the SRM object
srm = SRM(n_iter=n_iter, features=features)

# Fit the SRM data
print('Fitting SRM, may take a minute ...')
srm.fit(train_data)

print('SRM has been fit')

Fitting SRM, may take a minute ...
SRM has been fit


In [11]:
s = srm.transform(test_data)

In [12]:
print(s[0].shape)

(30, 592)


SRM should improve ISC, since it brings out stimulus-related variance.

In [75]:
test_data[0].shape

(1573, 592)

In [15]:
# Reorganize the data for run 1 of each condition (I, 8B, 2B, 1B)
orig_data = np.zeros((len(sub),test_data[0].shape[0],no_TRs,no_conds))
for s in range(len(test_data)):
  orig_data[s,:,:,0] = test_data[s][:,:no_TRs] # Intact
  orig_data[s,:,:,1] = test_data[s][:,no_TRs:no_TRs*2] # 8B
  orig_data[s,:,:,2] = test_data[s][:,no_TRs*2:no_TRs*3] # 2B
  orig_data[s,:,:,3] = test_data[s][:,no_TRs*3:] # 1B
# orig_data is subjects x voxels x TRs
# need it to be TRs x voxels x subjects for ISC computation
orig_data_T = np.transpose(orig_data, axes=(2,1,0,3))
print(orig_data_T.shape)
avg_data = np.average(orig_data_T,axis=1)
avg_data = np.reshape(avg_data,(no_TRs,1,len(sub),no_conds))
print(avg_data.shape)

orig_ISC_vals = []
for c in range(1):#no_conds):
  these_ISC_vals = isc(avg_data[:,:,:,c])
  print(these_ISC_vals)


#print(np.transpose(orig_data[:,:,:,0], axes=(2,1,0)).shape)




#orig_ISC_vals = []
#SRM_ISC_vals = []

(148, 1573, 4, 4)
(148, 1, 4, 4)
[[1.]
 [1.]
 [1.]
 [1.]]
